In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_dataset = pd.read_csv("/kaggle/input/titanic/train.csv", index_col='PassengerId')
test_dataset = pd.read_csv("/kaggle/input/titanic/test.csv",index_col='PassengerId')

In [3]:
xtrain = train_dataset.iloc[:, 1:]
ytrain = train_dataset.iloc[:, 0]

xtest = test_dataset.iloc[:, :]

In [4]:
print (xtest)

             Pclass                                          Name     Sex  \
PassengerId                                                                 
892               3                              Kelly, Mr. James    male   
893               3              Wilkes, Mrs. James (Ellen Needs)  female   
894               2                     Myles, Mr. Thomas Francis    male   
895               3                              Wirz, Mr. Albert    male   
896               3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   
...             ...                                           ...     ...   
1305              3                            Spector, Mr. Woolf    male   
1306              1                  Oliva y Ocana, Dona. Fermina  female   
1307              3                  Saether, Mr. Simon Sivertsen    male   
1308              3                           Ware, Mr. Frederick    male   
1309              3                      Peter, Master. Michael J    male   

In [5]:
print (xtrain)

             Pclass                                               Name  \
PassengerId                                                              
1                 3                            Braund, Mr. Owen Harris   
2                 1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
3                 3                             Heikkinen, Miss. Laina   
4                 1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
5                 3                           Allen, Mr. William Henry   
...             ...                                                ...   
887               2                              Montvila, Rev. Juozas   
888               1                       Graham, Miss. Margaret Edith   
889               3           Johnston, Miss. Catherine Helen "Carrie"   
890               1                              Behr, Mr. Karl Howell   
891               3                                Dooley, Mr. Patrick   

                Sex   Age  SibSp  Par

In [6]:
# Unnecessary columns
xtrain=xtrain.drop(['Name', 'Ticket', 'Cabin'], axis=1)
xtest=xtest.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [7]:
xtest

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34.5,0,0,7.8292,Q
893,3,female,47.0,1,0,7.0000,S
894,2,male,62.0,0,0,9.6875,Q
895,3,male,27.0,0,0,8.6625,S
896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
1305,3,male,NaN,0,0,8.0500,S
1306,1,female,39.0,0,0,108.9000,C
1307,3,male,38.5,0,0,7.2500,S


In [8]:
xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C
3,3,female,26.0,0,0,7.9250,S
4,1,female,35.0,1,0,53.1000,S
5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,S
888,1,female,19.0,0,0,30.0000,S
889,3,female,NaN,1,2,23.4500,S


In [9]:
xtrain.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [10]:
xtest.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [11]:
# Missing data in age and embarked in xtrain and age in xtest is replaced by the most frequent
from sklearn.impute import SimpleImputer
imputer2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer2.fit(xtrain.iloc[:, [2,6]])
xtrain.iloc[:, [2,6]] = imputer2.transform(xtrain.iloc[:, [2,6]])
imputer2.fit(xtest.iloc[:, [2]])
xtest.iloc[:, [2]] = imputer2.transform(xtest.iloc[:, [2]])

In [12]:
# Missing data in fare in xtest is replaced by the mean
imputer2 = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer2.fit(xtest.iloc[:, [5]])
xtest.iloc[:, [5]] = imputer2.transform(xtest.iloc[:, [5]])

In [13]:
print (xtrain)

             Pclass     Sex   Age  SibSp  Parch     Fare Embarked
PassengerId                                                      
1                 3    male  22.0      1      0   7.2500        S
2                 1  female  38.0      1      0  71.2833        C
3                 3  female  26.0      0      0   7.9250        S
4                 1  female  35.0      1      0  53.1000        S
5                 3    male  35.0      0      0   8.0500        S
...             ...     ...   ...    ...    ...      ...      ...
887               2    male  27.0      0      0  13.0000        S
888               1  female  19.0      0      0  30.0000        S
889               3  female  24.0      1      2  23.4500        S
890               1    male  26.0      0      0  30.0000        C
891               3    male  32.0      0      0   7.7500        Q

[891 rows x 7 columns]


In [14]:
print (xtest)

             Pclass     Sex   Age  SibSp  Parch      Fare Embarked
PassengerId                                                       
892               3    male  34.5      0      0    7.8292        Q
893               3  female  47.0      1      0    7.0000        S
894               2    male  62.0      0      0    9.6875        Q
895               3    male  27.0      0      0    8.6625        S
896               3  female  22.0      1      1   12.2875        S
...             ...     ...   ...    ...    ...       ...      ...
1305              3    male  21.0      0      0    8.0500        S
1306              1  female  39.0      0      0  108.9000        C
1307              3    male  38.5      0      0    7.2500        S
1308              3    male  21.0      0      0    8.0500        S
1309              3    male  21.0      1      1   22.3583        C

[418 rows x 7 columns]


In [15]:
xtrain.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [16]:
xtest.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtrain.iloc[:, [2,5]] = sc.fit_transform(xtrain.iloc[:, [2,5]])
xtest.iloc[:, [2,5]] = sc.transform(xtest.iloc[:, [2,5]])

In [18]:
print (xtrain)

             Pclass     Sex       Age  SibSp  Parch      Fare Embarked
PassengerId                                                           
1                 3    male -0.497793      1      0 -0.502445        S
2                 1  female  0.715048      1      0  0.786845        C
3                 3  female -0.194583      0      0 -0.488854        S
4                 1  female  0.487640      1      0  0.420730        S
5                 3    male  0.487640      0      0 -0.486337        S
...             ...     ...       ...    ...    ...       ...      ...
887               2    male -0.118780      0      0 -0.386671        S
888               1  female -0.725201      0      0 -0.044381        S
889               3  female -0.346188      1      2 -0.176263        S
890               1    male -0.194583      0      0 -0.044381        C
891               3    male  0.260233      0      0 -0.492378        Q

[891 rows x 7 columns]


In [19]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
encoder = OneHotEncoder()
ct = ColumnTransformer(transformers=[('encoder', encoder, [1,6])], remainder='passthrough')
xtrain=np.array(ct.fit_transform(xtrain))
xtest=np.array(ct.transform(xtest))

In [20]:
from sklearn.model_selection import train_test_split
xtrain, xtest1, ytrain, ytest = train_test_split (xtrain, ytrain, test_size=0.2)

In [21]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf')
classifier.fit(xtrain, ytrain)

SVC()

In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
ypred1=classifier.predict(xtest1)
matrix = confusion_matrix (ytest, ypred1)
print(matrix)
print (accuracy_score(ytest, ypred1))

[[100  11]
 [ 16  52]]
0.8491620111731844


In [23]:
print(accuracy_score (ytrain, classifier.predict(xtrain)))

0.8258426966292135


That is a good indicator that the model isn't overfitting as the score of the training set isn't more than the test set

In [24]:
TP = matrix[1][1]
TN = matrix[0][0]
FP = matrix[0][1]
FN = matrix[1][0]
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1_score = 2 * Precision * Recall / (Precision + Recall)

In [25]:
print (Accuracy)
print (Precision)
print (Recall)
print (F1_score)

0.8491620111731844
0.8253968253968254
0.7647058823529411
0.7938931297709922


In [26]:
ypred = classifier.predict(xtest)

In [27]:
len(ypred)

418

In [28]:
import csv

with open('results.csv', 'w') as csvfile:
    fieldnames = ['PassengerId', 'Survived']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range (0 ,len(ypred)):
        ID=892+i
        writer.writerow({'PassengerId' : ID, 'Survived' : ypred[i]})